# **Sarvam Translate API Tutorial**

This notebook demonstrates how to use the **Sarvam translate API** to translate texts/paragraphs from one language to another. The API supports additional features such as transliteration(a type of conversion of a text from one script to another that involves swapping letters), output_Script and gender .


## **Table of Contents**  

<p>1. <a href="#installation">Installation</a></p>  
<p>2. <a href="#authentication">Authentication</a></p>  
<p>3. <a href="#basic-usage">Basic Usage</a></p>  
<p>4. <a href="#modes">Modes</a></p>  
<p>5. <a href="#advanced-features">Advanced Features</a></p>  
   <p>&nbsp;&nbsp;&nbsp;&nbsp;5.1 <a href="#numeral-format">Numeral Format</a></p>  
   <p>&nbsp;&nbsp;&nbsp;&nbsp;5.2 <a href="#transliteration">Transliteration</a></p>  
   <p>&nbsp;&nbsp;&nbsp;&nbsp;5.3 <a href="#speaker-gender">Speaker Gender</a></p>  
<p>6. <a href="#error-handling">Error Handling</a></p>  
<p>7. <a href="#conclusion">Conclusion</a></p>  





## **Step 1. Installation**

Before you begin, ensure you have the necessary Python libraries installed. Run the following commands to install the required packages:

In [1]:
!pip install -Uqq sarvamai

In [2]:
from sarvamai import SarvamAI

## **Step 2: Authentication**


To use the Saaras API, you need an API subscription key. Follow these steps to set up your API key:

1. **Obtain your API key**: If you don’t have an API key, sign up on the [Sarvam AI Dashboard](https://dashboard.sarvam.ai/) to get one.
2. **Replace the placeholder key**: In the code below, replace "YOUR_SARVAM_AI_API_KEY" with your actual API key.

In [3]:
SARVAM_API_KEY = "YOUR_SARVAM_API_KEY"

## **Step 3: Basic Usage**


### **Step 3.1: Read the Document**

We have two sample documents under the `data` folder:  
- `sample1.txt` contains an essay on *The Impact of Artificial Intelligence on Society* in English.  
- `sample1.txt` contains an essay on *The Impact of Artificial Intelligence on Society* in Hindi.  

In [4]:
def read_file(file_path, lang_name):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            # Read the first 5 lines
            lines = [next(file) for _ in range(5)]
            print(f"=== {lang_name} Text (First Few Lines) ===")
            print("".join(lines))  # Print first few lines

            # Read the remaining content
            remaining_text = file.read()

            # Combine all text
            full_doc = "".join(lines) + remaining_text

            # Count total characters
            total_chars = len(full_doc)
            print(f"\nTotal number of characters in {lang_name} file:", total_chars)

            return full_doc
    except FileNotFoundError:
        print(f"Error: {file_path} not found.")
        return None
    except Exception as e:
        print(f"An error occurred while reading {file_path}: {e}")
        return None

In [5]:
# Read English and Hindi documents
english_doc = read_file("data/sample1.txt", "English")
hindi_doc = read_file("data/sample2.txt", "Hindi")

=== English Text (First Few Lines) ===
The Impact of Artificial Intelligence on Society

Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century, revolutionizing various aspects of human life. From healthcare to finance, transportation to education, AI is reshaping industries, automating processes, and augmenting human capabilities. While AI presents numerous benefits, it also raises ethical, economic, and societal concerns that must be carefully navigated to ensure its responsible integration into society.

One of the most profound impacts of AI is in the field of healthcare. AI-powered diagnostic tools enable early disease detection, improving patient outcomes and reducing healthcare costs. Machine learning algorithms analyze medical data to identify patterns that humans may overlook, aiding in personalized treatment plans. Robotic surgeries and AI-driven drug discoveries have further enhanced the efficiency of medical procedures. H

### **Step 3.2: Split the text into chunks of at most 1000 characters** 

Since the API has a restriction of 1000 characters per request, we need to split the text accordingly.

In [6]:
def chunk_text(text, max_length=1000):
    """Splits text into chunks of at most max_length characters while preserving word boundaries."""
    chunks = []

    while len(text) > max_length:
        split_index = text.rfind(" ", 0, max_length)  # Find the last space within limit
        if split_index == -1:
            split_index = max_length  # No space found, force split at max_length

        chunks.append(text[:split_index].strip())  # Trim spaces before adding
        text = text[split_index:].lstrip()  # Remove leading spaces for the next chunk

    if text:
        chunks.append(text.strip())  # Add the last chunk

    return chunks

In [7]:
# Split the text
english_text_chunks = chunk_text(english_doc)

# Display chunk info
print(f"Total Chunks: {len(english_text_chunks)}")
for i, chunk in enumerate(
    english_text_chunks[:3], 1
):  # Show only first 3 chunks for preview
    print(f"\n=== Chunk {i} (Length: {len(chunk)}) ===\n{chunk}")

Total Chunks: 2

=== Chunk 1 (Length: 993) ===
The Impact of Artificial Intelligence on Society

Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century, revolutionizing various aspects of human life. From healthcare to finance, transportation to education, AI is reshaping industries, automating processes, and augmenting human capabilities. While AI presents numerous benefits, it also raises ethical, economic, and societal concerns that must be carefully navigated to ensure its responsible integration into society.

One of the most profound impacts of AI is in the field of healthcare. AI-powered diagnostic tools enable early disease detection, improving patient outcomes and reducing healthcare costs. Machine learning algorithms analyze medical data to identify patterns that humans may overlook, aiding in personalized treatment plans. Robotic surgeries and AI-driven drug discoveries have further enhanced the efficiency of medical proce

In [8]:
# Split the text
hindi_text_chunks = chunk_text(english_doc)

# Display chunk info
print(f"Total Chunks: {len(hindi_text_chunks)}")
for i, chunk in enumerate(
    hindi_text_chunks[:3], 1
):  # Show only first 3 chunks for preview
    print(f"\n=== Chunk {i} (Length: {len(chunk)}) ===\n{chunk}")

Total Chunks: 2

=== Chunk 1 (Length: 993) ===
The Impact of Artificial Intelligence on Society

Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century, revolutionizing various aspects of human life. From healthcare to finance, transportation to education, AI is reshaping industries, automating processes, and augmenting human capabilities. While AI presents numerous benefits, it also raises ethical, economic, and societal concerns that must be carefully navigated to ensure its responsible integration into society.

One of the most profound impacts of AI is in the field of healthcare. AI-powered diagnostic tools enable early disease detection, improving patient outcomes and reducing healthcare costs. Machine learning algorithms analyze medical data to identify patterns that humans may overlook, aiding in personalized treatment plans. Robotic surgeries and AI-driven drug discoveries have further enhanced the efficiency of medical proce

### **Step 3.3: Setting up the API Endpoint** 

There are three main types of translations we support:  

1️⃣ **English to Indic** 🏛 → Translating from **English to Indian languages** (e.g., *"Invoice total is $3,450.75." → "इनवॉइस कुल राशि $3,450.75 है।"*)  

2️⃣ **Indic to English** 🌍 → Converting **Indian languages to English** (e.g., *"आपका ऑर्डर सफलतापूर्वक दर्ज किया गया है।" → "Your order has been successfully placed."*)  

3️⃣ **Indic to Indic** 🔄 → Translating **between Indian languages** (e.g., *Hindi → Tamil, Bengali → Marathi*).  


In [9]:
# Initialize SarvamAI

from sarvamai import SarvamAI

client = SarvamAI(api_subscription_key=SARVAM_API_KEY)

### **Indic to Eng Translation**  

In [10]:
translated_texts = []
for idx, chunk in enumerate(hindi_text_chunks):
    response = client.text.translate(
        input=chunk,
        source_language_code="hi-IN",
        target_language_code="en-IN",
        speaker_gender="Male",
        mode="classic-colloquial",
        model="mayura:v1",
        enable_preprocessing=False,
    )

    translated_text = response.translated_text
    print(f"\n=== Translated Chunk {idx + 1} ===\n{translated_text}\n")
    translated_texts.append(translated_text)

# Combine all translated chunks
final_translation = "\n".join(translated_texts)
print("\n=== Final Translated Text ===")
print(final_translation)


=== Translated Chunk 1 ===
The Impact of Artificial Intelligence on Society

Artificial Intelligence (AI) is revolutionizing various aspects of human life in the 21st century, from healthcare to finance, transportation to education. AI is reshaping industries, automating processes, and augmenting human capabilities. While it brings numerous benefits, it also raises ethical, economic, and societal concerns that need to be carefully navigated.

One of the most profound impacts of AI is in the field of healthcare. AI-powered diagnostic tools enable early disease detection, improving patient outcomes and reducing healthcare costs. Machine learning algorithms analyze medical data to identify patterns that humans may overlook, aiding in personalized treatment plans. Robotic surgeries and AI-driven drug discoveries have revolutionized medical procedures.


=== Translated Chunk 2 ===
The use of healthcare raises significant concerns pertaining to data privacy, security, and potential biases i

### **Indic to Indic Translation**  

In [11]:
translated_texts = []
for idx, chunk in enumerate(hindi_text_chunks):
    response = client.text.translate(
        input=chunk,
        source_language_code="hi-IN",
        target_language_code="bn-IN",
        speaker_gender="Male",
        mode="classic-colloquial",
        model="mayura:v1",
        enable_preprocessing=False,
    )

    translated_text = response.translated_text
    print(f"\n=== Translated Chunk {idx + 1} ===\n{translated_text}\n")
    translated_texts.append(translated_text)

# Combine all translated chunks
final_translation = "\n".join(translated_texts)
print("\n=== Translated Text Chunks in Bengali ===")
print(final_translation)


=== Translated Chunk 1 ===
Artificial Intelligence-এর সমাজে প্রভাব

21st century-তে Artificial Intelligence (AI) মানুষের জীবনের বিভিন্ন দিক বদলে দিচ্ছে। Healthcare থেকে finance, transportation থেকে education, AI industry-গুলোকে নতুন করে গড়ে তুলছে আর process-গুলোকে automate করছে। AI অনেক benefit দিলেও, ethical, economic, আর societal concern-ও তৈরি করছে, যেগুলো  carefully navigate করা দরকার।

AI-এর একটা বড় impact healthcare sector-এ। AI-powered diagnostic tools-এর মাধ্যমে disease detection অনেক আগেই করা যায়, patient-দের outcome ভালো হয়, আর healthcare cost-ও কমে। Machine learning algorithm-গুলো medical data analyze করে এমন pattern খুঁজে বের করে, যেটা মানুষ overlook করতে পারে। এতে personalized treatment plan-এ help হয়। Robotic surgery আর AI-driven drug discovery, medical procedure-কে revolutionize করেছে।


=== Translated Chunk 2 ===
Healthcare use করার সময় data privacy, security, আর medical decision making-এ potential bias-এর মতো  concern-গুলো  গুরুত্বপূর্ণ।


=== Translated Text Chunk

### **Eng to Indic Translation**  

In [12]:
translated_texts = []
for idx, chunk in enumerate(english_text_chunks):
    response = client.text.translate(
        input=chunk,
        source_language_code="en-IN",
        target_language_code="pa-IN",
        speaker_gender="Male",
        mode="classic-colloquial",
        model="mayura:v1",
        enable_preprocessing=False,
    )

    translated_text = response.translated_text
    print(f"\n=== Translated Chunk {idx + 1} ===\n{translated_text}\n")
    translated_texts.append(translated_text)

# Combine all translated chunks
final_translation = "\n".join(translated_texts)
print("\n=== Final Translated Text ===")
print(final_translation)


=== Translated Chunk 1 ===
Artificial Intelligence ਦਾ ਸਮਾਜ 'ਤੇ ਅਸਰ

Artificial Intelligence (AI) 21ਵੀਂ ਸਦੀ ਦੀਆਂ ਸਭ ਤੋਂ ਵੱਡੀਆਂ technologies ਵਿੱਚੋਂ ਇੱਕ ਬਣ ਗਈ ਹੈ, ਜੋ ਮਨੁੱਖੀ ਜੀਵਨ ਦੇ ਕਈ ਪਹਿਲੂਆਂ ਨੂੰ ਬਦਲ ਰਹੀ ਹੈ। Healthcare ਤੋਂ ਲੈ ਕੇ finance, transportation ਤੋਂ ਲੈ ਕੇ education ਤੱਕ, AI industries ਨੂੰ ਬਦਲ ਰਹੀ ਹੈ, processes ਨੂੰ automate ਕਰ ਰਹੀ ਹੈ, ਅਤੇ ਮਨੁੱਖੀ ਯੋਗਤਾਵਾਂ ਨੂੰ ਵਧਾ ਰਹੀ ਹੈ। AI ਦੇ ਕਈ ਫਾਇਦੇ ਹਨ, ਪਰ ਇਸਦੇ ਨਾਲ ethical, economic, ਅਤੇ ਸਮਾਜਿਕ ਚਿੰਤਾਵਾਂ ਵੀ ਜੁੜੀਆਂ ਹਨ ਜਿਨ੍ਹਾਂ ਨੂੰ ਧਿਆਨ ਨਾਲ ਸਮਾਜ ਵਿੱਚ ਇਸਦੀ ਜ਼ਿੰਮੇਵਾਰ integration ਲਈ ਦੇਖਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ।

AI ਦਾ ਸਭ ਤੋਂ ਵੱਡਾ ਪ੍ਰਭਾਵ healthcare ਦੇ ਖੇਤਰ ਵਿੱਚ ਹੈ। AI-powered diagnostic tools ਨਾਲ ਬਿਮਾਰੀ ਦਾ ਜਲਦੀ ਪਤਾ ਲੱਗ ਸਕਦਾ ਹੈ, ਜਿਸ ਨਾਲ ਮਰੀਜ਼ਾਂ ਦੇ ਨਤੀਜੇ ਸੁਧਰਦੇ ਹਨ ਅਤੇ healthcare costs ਘੱਟ ਹੁੰਦੇ ਹਨ। Machine learning algorithms medical data ਦਾ ਵਿਸ਼ਲੇਸ਼ਣ ਕਰਕੇ patterns ਲੱਭਦੇ ਹਨ ਜੋ ਮਨੁੱਖ ਨਹੀਂ ਦੇਖ ਸਕਦੇ, ਜਿਸ ਨਾਲ personalized treatment plans ਵਿੱਚ ਮਦਦ ਮਿਲਦੀ ਹੈ। Robotic surgeries ਅਤੇ AI-driven drug discoveries ਨੇ medical procedures ਦੀ efficiency ਵਧਾ ਦਿੱਤੀ ਹੈ। ਪਰ, AI ਦਾ integ

## **Step 4: Translation Modes**  

### **Translation Modes & Differences**  

1️⃣ **Formal** – Highly professional, uses **pure Hindi** (e.g., *"कुल राशि", "देय है"*). Suitable for **official documents, legal papers, and corporate communication**.  

2️⃣ **Classic-Colloquial** – Balanced mix of **Hindi & English**, slightly informal (e.g., *"कुल जोड़", "देना होगा"*). Ideal for **business emails, customer support, and semi-formal communication**.  

3️⃣ **Modern-Colloquial** – **Hinglish, casual, and direct** (e.g., *"Invoice total", "due है", "contact करो"*). Best for **chatbots, social media, and casual conversations**.  

**📌 Rule of Thumb:**  
- **Use Formal for official content** 🏛  
- **Use Classic-Colloquial for general communication** 💬  
- **Use Modern-Colloquial for everyday conversations** 🚀  



In [13]:
# To highlight the difference between the models lets use the below example.
full_text = (
    "The invoice total is $3,450.75, due by 15th March 2025. Contact us at support@example.com for queries. "
    "Order #987654321 was placed on 02/29/2024. Your tracking ID is TRK12345678."
)

### **Step 4.1: Classic Colloquial**  

In [14]:
response = client.text.translate(
    input=full_text,
    source_language_code="en-IN",
    target_language_code="hi-IN",
    speaker_gender="Male",
    mode="classic-colloquial",
    model="mayura:v1",
    enable_preprocessing=False,
)
translated_text = response.translated_text
print("\n=== Translated Text ===\n", translated_text)


=== Translated Text ===
 Invoice का कुल दाम $3,450.75 है, जो 15th March 2025 तक देना होगा। पूछताछ के लिए support@example.com पर हमसे contact करें। Order #987654321 02/29/2024 को लगाया गया था। आपका tracking ID TRK12345678 है।


### **Step 4.2: Formal**  

In [15]:
response = client.text.translate(
    input=full_text,
    source_language_code="en-IN",
    target_language_code="hi-IN",
    speaker_gender="Male",
    mode="formal",
    model="mayura:v1",
    enable_preprocessing=False,
)
translated_text = response.translated_text
print("\n=== Translated Text ===\n", translated_text)


=== Translated Text ===
 इनवॉइस कुल राशि $3,450.75 है, जो 15 मार्च 2025 तक देय है। प्रश्नों के लिए support@example.com पर हमसे संपर्क करें। आदेश #987654321 02/29/2024 को दिया गया था। आपकी ट्रैकिंग आईडी टीआरके12345678 है।


### **Step 4.3: Modern Colloquial**  

In [16]:
response = client.text.translate(
    input=full_text,
    source_language_code="en-IN",
    target_language_code="hi-IN",
    speaker_gender="Male",
    mode="modern-colloquial",
    model="mayura:v1",
    enable_preprocessing=False,
)
translated_text = response.translated_text
print("\n=== Translated Text ===\n", translated_text)


=== Translated Text ===
 Invoice total $3,450.75 है, 15th March 2025 तक due है। Queries के लिए support@example.com पर contact कीजिए। Order #987654321 02/29/2024 को place हुआ था। आपका tracking ID TRK12345678 है।


## **Step 5: Advanced Features**  





### **Step 5.1: Speaker Gender**  

The translation model supports **Male** and **Female** speaker options, which impact the tone and style of the output.  

1️⃣ **Male Voice** 🔵 

2️⃣ **Female Voice** 🔴 


### **Female**  

In [17]:
response = client.text.translate(
    input=full_text,
    source_language_code="en-IN",
    target_language_code="hi-IN",
    speaker_gender="Female",
    mode="modern-colloquial",
    model="mayura:v1",
    enable_preprocessing=False,
)
translated_text = response.translated_text
print("\n=== Translated Text ===\n", translated_text)


=== Translated Text ===
 Invoice total $3,450.75 है, 15th March 2025 तक due है। Queries के लिए support@example.com पर contact कीजिए। Order #987654321 02/29/2024 को place हुआ था। आपका tracking ID TRK12345678 है।


### **Male**  

In [18]:
response = client.text.translate(
    input=full_text,
    source_language_code="en-IN",
    target_language_code="hi-IN",
    speaker_gender="Male",
    mode="modern-colloquial",
    model="mayura:v1",
    enable_preprocessing=False,
)
translated_text = response.translated_text
print("\n=== Translated Text ===\n", translated_text)


=== Translated Text ===
 Invoice total $3,450.75 है, 15th March 2025 तक due है। Queries के लिए support@example.com पर contact कीजिए। Order #987654321 02/29/2024 को place हुआ था। आपका tracking ID TRK12345678 है।


### **Step 5.2: Numerals Format Feature**  

The `numerals_format` parameter controls how numbers appear in the translation. It has two options:  

1️⃣ **International (Default)** 🌍 → Uses standard **0-9** numerals.  
✅ Example: *"मेरा phone number है: 9840950950."*  
✅ Best for **universally understood content, technical documents, and modern usage**.  

2️⃣ **Native** 🔡 → Uses **language-specific** numerals.  
✅ Example: *"मेरा phone number है: ९८४०९५०९५०."*  
✅ Ideal for **traditional texts, cultural adaptation, and regional content**.  

**📌 When to Use What?**  
- Use **International** for **wider readability and digital content** 📱  
- Use **Native** for **localized, heritage-focused, and print media content** 📖  


### **Native**  

In [19]:
response = client.text.translate(
    input=full_text,
    source_language_code="en-IN",
    target_language_code="hi-IN",
    speaker_gender="Male",
    mode="modern-colloquial",
    model="mayura:v1",
    enable_preprocessing=False,
    numerals_format="native",
)
translated_text = response.translated_text
print("\n=== Translated Text ===\n", translated_text)


=== Translated Text ===
 Invoice total $३,४५०.७५ है, १५th March २०२५ तक due है। Queries के लिए support@example.com पर contact कीजिए। Order #९८७६५४३२१ ०२/२९/२०२४ को place हुआ था। आपका tracking ID TRK१२३४५६७८ है।


### **International**  

In [20]:
response = client.text.translate(
    input=full_text,
    source_language_code="en-IN",
    target_language_code="hi-IN",
    speaker_gender="Male",
    mode="modern-colloquial",
    model="mayura:v1",
    enable_preprocessing=False,
    numerals_format="international",
)
translated_text = response.translated_text
print("\n=== Translated Text ===\n", translated_text)


=== Translated Text ===
 Invoice total $3,450.75 है, 15th March 2025 तक due है। Queries के लिए support@example.com पर contact कीजिए। Order #987654321 02/29/2024 को place हुआ था। आपका tracking ID TRK12345678 है।


### **Step 5.3: Numerals Format Feature**  

The `output_script` parameter controls how the translated text is **transliterated**, i.e., how it appears in different scripts while keeping pronunciation intact.  

### **Transliteration Options:**  

1️⃣ **Default (null)** – No transliteration applied.  
✅ Example: *"आपका Rs. 3000 का EMI pending है।"*  
✅ Best for **modern, mixed-language content**.  

2️⃣ **Roman** – Converts the output into **Romanized Hindi**.  
✅ Example: *"aapka Rs. 3000 ka EMI pending hai."*  
✅ Ideal for **users who can speak but not read native scripts**.  

3️⃣ **Fully-Native** – Uses **formal native script transliteration**.  
✅ Example: *"आपका रु. 3000 का ई.एम.ऐ. पेंडिंग है।"*  
✅ Best for **official documents and structured writing**.  

4️⃣ **Spoken-Form-in-Native** – Uses **native script but mimics spoken style**.  
✅ Example: *"आपका थ्री थाउजेंड रूपीस का ईएमअइ पेंडिंग है।"*  
✅ Ideal for **voice assistants, conversational AI, and informal speech**.  

### **📌 When to Use What?**  
- **Default** – For natural, mixed-language modern writing ✍️  
- **Roman** – For users unfamiliar with native scripts 🔤  
- **Fully-Native** – For formal, structured translations 🏛  
- **Spoken-Form-in-Native** – For casual speech and voice applications 🎙  


In [21]:
response = client.text.translate(
    input=full_text,
    source_language_code="en-IN",
    target_language_code="hi-IN",
    speaker_gender="Male",
    mode="modern-colloquial",
    model="mayura:v1",
    enable_preprocessing=False,
    output_script="roman",
    numerals_format="international",
)
translated_text = response.translated_text
print("\n=== Translated Text ===\n", translated_text)


=== Translated Text ===
 Invoice total $3,450.75 hai, 15th March 2025 tak due hai. Queries ke liye support@example.com par contact kijiye. Order #987654321 02/29/2024 ko place hua tha. Aapka tracking ID TRK12345678 hai.


In [22]:
response = client.text.translate(
    input=full_text,
    source_language_code="en-IN",
    target_language_code="hi-IN",
    speaker_gender="Male",
    mode="modern-colloquial",
    model="mayura:v1",
    enable_preprocessing=False,
    output_script="spoken-form-in-native",
    numerals_format="international",
)
translated_text = response.translated_text
print("\n=== Translated Text ===\n", translated_text)


=== Translated Text ===
 इनवॉइस टोटल थ्री थाउजेंड, फोर हंड्रेड एंड फिफ्टी पॉइंट सेवन फाइव यू एस डॉलर्स है, फिफ्टीन्थ मार्च ट्वेंटी ट्वेंटी फाइव तक ड्यू है। क्वेरीज़ के लिए सपोर्ट एट द रेट एक्जाम्पल डॉट कॉम पर कांटेक्ट कीजिए। ऑर्डर हैश नाइन एइट सेवन, सिक्स फाइव फोर, थ्री टू वन ट्वेंटी नाइंथ फ़रवरी ट्वेंटी ट्वेंटी फोर को प्लेस हुआ था। आपका ट्रैकिंग आईडी टी आर के वन टू थ्री, फोर फाइव सिक्स, सेवन एइट है।


In [23]:
response = client.text.translate(
    input=full_text,
    source_language_code="en-IN",
    target_language_code="hi-IN",
    speaker_gender="Male",
    mode="modern-colloquial",
    model="mayura:v1",
    enable_preprocessing=False,
    output_script="fully-native",
    numerals_format="international",
)
translated_text = response.translated_text
print("\n=== Translated Text ===\n", translated_text)


=== Translated Text ===
 इनवॉइस टोटल $3,450.75 है, 15थ मार्च 2025 तक ड्यू है। क्वेरीज़ के लिए support@example.com पर कॉन्टैक्ट कीजिए। ऑर्डर #987654321 02/29/2024 को प्लेस हुआ था। आपका ट्रैकिंग आईडी टीआरके12345678 है।




## **6️. Error Handling**  

You may encounter these errors while using the API:  

- **403 Forbidden** (`invalid_api_key_error`)  
  - Cause: Invalid API key.  
  - Solution: Use a valid API key from the [Sarvam AI Dashboard](https://dashboard.sarvam.ai/).  

- **429 Too Many Requests** (`insufficient_quota_error`)  
  - Cause: Exceeded API quota.  
  - Solution: Check your usage, upgrade if needed, or implement exponential backoff when retrying.  

- **500 Internal Server Error** (`internal_server_error`)  
  - Cause: Issue on our servers.  
  - Solution: Try again later. If persistent, contact support.  

- **400 Bad Request** (`invalid_request_error`)  
  - Cause: Incorrect request formatting.  
  - Solution: Verify your request structure and parameters.  



### **7. Additional Resources**

For more details, refer to the our official documentation and we are always there to support and help you on our Discord Server:

- **Documentation**: [docs.sarvam.ai](https://docs.sarvam.ai)  
- **Community**: [Join the Discord Community](https://discord.gg/hTuVuPNF)

---

### **8. Final Notes**

- Keep your API key secure.
- Use clear audio for best results.
- Explore advanced features like diarization and translation.

**Keep Building!** 🚀